# Робота з моделями родини Meta

## Вступ

У цьому уроці ми розглянемо:

- Дослідження двох основних моделей родини Meta — Llama 3.1 та Llama 3.2
- Розуміння сценаріїв використання кожної моделі
- Приклад коду, який демонструє унікальні можливості кожної моделі

## Родина моделей Meta

У цьому уроці ми розглянемо 2 моделі з родини Meta, або "Llama Herd" — Llama 3.1 та Llama 3.2

Ці моделі мають різні варіанти та доступні на маркетплейсі Github Model. Більше інформації про використання Github Models для [прототипування з AI-моделями](https://docs.github.com/en/github-models/prototyping-with-ai-models?WT.mc_id=academic-105485-koreyst).

Варіанти моделей:
- Llama 3.1 — 70B Instruct
- Llama 3.1 — 405B Instruct
- Llama 3.2 — 11B Vision Instruct
- Llama 3.2 — 90B Vision Instruct

*Примітка: Llama 3 також доступна на Github Models, але в цьому уроці вона не розглядається*


## Llama 3.1

З 405 мільярдами параметрів, Llama 3.1 належить до категорії відкритих LLM.

Ця модель є оновленням попередньої версії Llama 3 і пропонує:

- Більше контекстне вікно — 128k токенів проти 8k токенів
- Більший максимум вихідних токенів — 4096 проти 2048
- Краща багатомовна підтримка — завдяки збільшенню кількості тренувальних токенів

Ці можливості дозволяють Llama 3.1 вирішувати складніші завдання при створенні GenAI-додатків, зокрема:
- Вбудований виклик функцій — можливість викликати зовнішні інструменти та функції поза межами LLM
- Краща продуктивність RAG — завдяки більшому контекстному вікну
- Генерація синтетичних даних — можливість створювати ефективні дані для задач, таких як донавчання


### Виклик вбудованих функцій

Llama 3.1 була донавчена для більш ефективного використання функцій або інструментів. Вона також має два вбудовані інструменти, які модель може розпізнати як необхідні для використання на основі запиту користувача. Ці інструменти:

- **Brave Search** — використовується для отримання актуальної інформації, наприклад, про погоду, шляхом веб-пошуку
- **Wolfram Alpha** — використовується для складніших математичних обчислень, тому немає потреби писати власні функції.

Ви також можете створювати власні інструменти, які LLM зможе викликати.

У наведеному нижче прикладі коду:

- Ми визначаємо доступні інструменти (brave_search, wolfram_alpha) у системному запиті.
- Надсилаємо користувацький запит про погоду в певному місті.
- LLM відповість викликом інструменту Brave Search, який виглядатиме так: `<|python_tag|>brave_search.call(query="Stockholm weather")`

*Примітка: У цьому прикладі здійснюється лише виклик інструменту. Якщо ви хочете отримати результати, потрібно створити безкоштовний обліковий запис на сторінці Brave API та визначити саму функцію.*


In [ ]:
%pip install azure-core
%pip install azure-ai-inference

In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import AssistantMessage, SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

token = os.environ["GITHUB_TOKEN"]
endpoint = "https://models.inference.ai.azure.com"
model_name = "meta-llama-3.1-405b-instruct"

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)


tool_prompt=f"""
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Environment: ipython
Tools: brave_search, wolfram_alpha
Cutting Knowledge Date: December 2023
Today Date: 23 July 2024

You are a helpful assistant<|eot_id|>
"""

messages = [
    SystemMessage(content=tool_prompt),
    UserMessage(content="What is the weather in Stockholm?"),

]

response = client.complete(messages=messages, model=model_name)

print(response.choices[0].message.content)





### Llama 3.2

Попри те, що Llama 3.1 є LLM, одне з її обмежень — це відсутність мультимодальності. Тобто, вона не може використовувати різні типи вхідних даних, наприклад, зображення як підказки та надавати відповіді. Ця можливість є однією з головних особливостей Llama 3.2. До цих можливостей також належать:

- Мультимодальність — здатна аналізувати як текстові, так і візуальні підказки
- Варіанти малого та середнього розміру (11B та 90B) — це забезпечує гнучкі варіанти розгортання,
- Варіанти лише для тексту (1B та 3B) — це дозволяє запускати модель на пристроях з обмеженими ресурсами або мобільних пристроях і забезпечує низьку затримку

Підтримка мультимодальності — це значний крок у світі моделей з відкритим кодом. Наведений нижче приклад коду приймає як зображення, так і текстову підказку, щоб отримати аналіз зображення від Llama 3.2 90B.

### Мультимодальна підтримка в Llama 3.2


In [ ]:
%pip install azure-core
%pip install azure-ai-inference

In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import (
    SystemMessage,
    UserMessage,
    TextContentItem,
    ImageContentItem,
    ImageUrl,
    ImageDetailLevel,
)
from azure.core.credentials import AzureKeyCredential

token = os.environ["GITHUB_TOKEN"]
endpoint = "https://models.inference.ai.azure.com"
model_name = "Llama-3.2-90B-Vision-Instruct"


client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = client.complete(
    messages=[
        SystemMessage(
            content="You are a helpful assistant that describes images in details."
        ),
        UserMessage(
            content=[
                TextContentItem(text="What's in this image?"),
                ImageContentItem(
                    image_url=ImageUrl.load(
                        image_file="sample.jpg",
                        image_format="jpg",
                        detail=ImageDetailLevel.LOW)
                ),
            ],
        ),
    ],
    model=model_name,
)

print(response.choices[0].message.content)

## Навчання не закінчується тут, продовжуйте свій шлях

Після завершення цього уроку перегляньте нашу [колекцію навчальних матеріалів з генеративного ШІ](https://aka.ms/genai-collection?WT.mc_id=academic-105485-koreyst), щоб і далі поглиблювати свої знання у сфері генеративного ШІ!



---

**Відмова від відповідальності**:  
Цей документ було перекладено за допомогою сервісу автоматичного перекладу [Co-op Translator](https://github.com/Azure/co-op-translator). Хоча ми прагнемо до точності, звертаємо вашу увагу, що автоматичний переклад може містити помилки або неточності. Оригінальний документ рідною мовою слід вважати авторитетним джерелом. Для отримання критично важливої інформації рекомендується професійний людський переклад. Ми не несемо відповідальності за будь-які непорозуміння або неправильне тлумачення, що виникли внаслідок використання цього перекладу.
